In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
import logging
import yaml
from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
settings = {'activation':'sigmoid', 'output_size': 17, 'loss':'binary_crossentropy'}
inception_v3 = {'name':'inception_v3', 'input_shape': (299,299,3), 'batch_size': 16}

In [3]:
# create data generators
def get_labels():
    with open('../label_maps.yml', 'r') as lablels:
        label_data = yaml.load(lablels)
    inv_label_map = label_data['inv_label_map']
    label_map = label_data['label_map']
    return label_map, inv_label_map

label_map, inv_label_map = get_labels()

In [4]:
def load_images_and_resize(img_shape, output_size):
    df_train = pd.read_csv('../data/train_v2.csv')
    logging.info('Loading images and resize to {}'.format(img_shape))
    x_train = []
    y_train = []
    for f, tags in tqdm(df_train.values, miniters=1000):
        img = cv2.imread('../data/train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        x_train.append(cv2.resize(img, img_shape))
        y_train.append(targets)
    y_train = np.array(y_train, np.uint8)
    x_train = np.array(x_train, np.uint8)
    
    x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                          y_train,
                                                          test_size=0.15,
                                                          random_state=42)
    
    return x_train, x_valid, y_train, y_valid

x_train, x_valid, y_train, y_valid = load_images_and_resize(inception_v3['input_shape'][:2],
                                          settings['output_size'])

nb_train_examples = len(x_train)
nb_valid_examples = len(x_valid)

100%|██████████| 40479/40479 [01:17<00:00, 521.54it/s]


In [5]:
import sys
print('Train data size: {:.2f} GB'.format(sys.getsizeof(x_train)/1e9))
print('input shape: {}'.format(x_train.shape))
print('Train data size: {:.2f} GB'.format(sys.getsizeof(x_valid)/1e9))
print('input shape: {}'.format(x_valid.shape))

Train data size: 9.23 GB
input shape: (34407, 299, 299, 3)
Train data size: 1.63 GB
input shape: (6072, 299, 299, 3)


In [6]:
train_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

train_generator_loaded = train_generator.flow(x_train, y_train,
                                              batch_size=inception_v3['batch_size'])
x_train = []

valid_generator = ImageDataGenerator(rescale=1. / 255)

valid_generator_loaded = valid_generator.flow(
        x_valid, y_valid,
        batch_size=inception_v3['batch_size'],
        shuffle=False)

x_valid = []

# Train top layer

In [7]:
file_path = '../models/{}_'.format(inception_v3['name']) + '{epoch:02d}_{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(file_path, monitor='val_loss',
                                   verbose=0, save_best_only=False, 
                                save_weights_only=False, mode='auto',
                                   period=1)

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

In [10]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(settings['output_size'], activation=settings['activation'])(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss=settings['loss'], metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator_loaded,
                    steps_per_epoch=nb_train_examples / inception_v3['batch_size'],
                    epochs=20, validation_data=valid_generator_loaded,
                    validation_steps = nb_valid_examples / inception_v3['batch_size'],
                    callbacks = [model_checkpoint, es], verbose=1)



# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)


Epoch 1/20
269/268 [==============================] - 650s - loss: 0.2029 - acc: 0.9253 - val_loss: 0.1764 - val_acc: 0.9297
Epoch 2/20
269/268 [==============================] - 622s - loss: 0.1508 - acc: 0.9419 - val_loss: 0.1492 - val_acc: 0.9421
Epoch 3/20
269/268 [==============================] - 623s - loss: 0.1426 - acc: 0.9454 - val_loss: 0.1419 - val_acc: 0.9461
Epoch 4/20
269/268 [==============================] - 623s - loss: 0.1388 - acc: 0.9471 - val_loss: 0.1332 - val_acc: 0.9491
Epoch 5/20
269/268 [==============================] - 623s - loss: 0.1351 - acc: 0.9483 - val_loss: 0.1288 - val_acc: 0.9511
Epoch 6/20
269/268 [==============================] - 623s - loss: 0.1337 - acc: 0.9490 - val_loss: 0.1367 - val_acc: 0.9479
Epoch 7/20
269/268 [==============================] - 623s - loss: 0.1321 - acc: 0.9499 - val_loss: 0.1231 - val_acc: 0.9534
Epoch 8/20
269/268 [==============================] - 622s - loss: 0.1312 - acc: 0.9504 - val_loss: 0.1250 - val_acc: 0.9523


In [11]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=settings['loss'])

0 input_3
1 conv2d_189
2 batch_normalization_189
3 activation_189
4 conv2d_190
5 batch_normalization_190
6 activation_190
7 conv2d_191
8 batch_normalization_191
9 activation_191
10 max_pooling2d_9
11 conv2d_192
12 batch_normalization_192
13 activation_192
14 conv2d_193
15 batch_normalization_193
16 activation_193
17 max_pooling2d_10
18 conv2d_197
19 batch_normalization_197
20 activation_197
21 conv2d_195
22 conv2d_198
23 batch_normalization_195
24 batch_normalization_198
25 activation_195
26 activation_198
27 average_pooling2d_19
28 conv2d_194
29 conv2d_196
30 conv2d_199
31 conv2d_200
32 batch_normalization_194
33 batch_normalization_196
34 batch_normalization_199
35 batch_normalization_200
36 activation_194
37 activation_196
38 activation_199
39 activation_200
40 mixed0
41 conv2d_204
42 batch_normalization_204
43 activation_204
44 conv2d_202
45 conv2d_205
46 batch_normalization_202
47 batch_normalization_205
48 activation_202
49 activation_205
50 average_pooling2d_20
51 conv2d_201
52 

In [12]:
model.fit_generator(train_generator_loaded,
                    steps_per_epoch=nb_train_examples / inception_v3['batch_size'],
                    epochs=30, validation_data=valid_generator_loaded,
                    validation_steps = nb_valid_examples / inception_v3['batch_size'],
                    callbacks = [model_checkpoint, es], verbose=1, initial_epoch = 12)

Epoch 13/30
269/268 [==============================] - 913s - loss: 0.1262 - val_loss: 0.1224
Epoch 14/30
269/268 [==============================] - 899s - loss: 0.1232 - val_loss: 0.1216
Epoch 15/30
269/268 [==============================] - 899s - loss: 0.1217 - val_loss: 0.1209
Epoch 16/30
269/268 [==============================] - 899s - loss: 0.1210 - val_loss: 0.1203
Epoch 17/30
269/268 [==============================] - 899s - loss: 0.1202 - val_loss: 0.1199
Epoch 18/30
269/268 [==============================] - 892s - loss: 0.1199 - val_loss: 0.1194
Epoch 19/30
269/268 [==============================] - 893s - loss: 0.1190 - val_loss: 0.1191
Epoch 20/30
269/268 [==============================] - 889s - loss: 0.1195 - val_loss: 0.1187
Epoch 21/30
  3/268 [..............................] - ETA: 784s - loss: 0.1168

KeyboardInterrupt: 

In [13]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=settings['loss'])

model.fit_generator(train_generator_loaded,
                    steps_per_epoch=nb_train_examples / inception_v3['batch_size'],
                    epochs=100, validation_data=valid_generator_loaded,
                    validation_steps = nb_valid_examples / inception_v3['batch_size'],
                    callbacks = [model_checkpoint, es], verbose=1, initial_epoch = 20)

Epoch 21/100
269/268 [==============================] - 897s - loss: 0.1176 - val_loss: 0.1163
Epoch 22/100
269/268 [==============================] - 885s - loss: 0.1137 - val_loss: 0.1146
Epoch 23/100
269/268 [==============================] - 886s - loss: 0.1137 - val_loss: 0.1130
Epoch 24/100
269/268 [==============================] - 889s - loss: 0.1121 - val_loss: 0.1116
Epoch 25/100
269/268 [==============================] - 886s - loss: 0.1102 - val_loss: 0.1105
Epoch 26/100
269/268 [==============================] - 889s - loss: 0.1100 - val_loss: 0.1096
Epoch 27/100
269/268 [==============================] - 882s - loss: 0.1085 - val_loss: 0.1089
Epoch 28/100
269/268 [==============================] - 896s - loss: 0.1078 - val_loss: 0.1082
Epoch 29/100
269/268 [==============================] - 902s - loss: 0.1063 - val_loss: 0.1072
Epoch 30/100
269/268 [==============================] - 902s - loss: 0.1062 - val_loss: 0.1069
Epoch 31/100
269/268 [============================

KeyboardInterrupt: 

# Train full model
- half model final line

Epoch 58/100
269/268 [==============================] - 902s - loss: 0.0961 - val_loss: 0.0994

In [13]:
# full model


#model = load_model('../models/inception_v3_full_28_0.09.hdf5')

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

file_path = '../models/{}_full_'.format(inception_v3['name']) + '{epoch:02d}_{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(file_path, monitor='val_loss',
                                   verbose=0, save_best_only=False, 
                                save_weights_only=False, mode='auto',
                                   period=1)

for layer in model.layers[:172]:
   layer.trainable = True
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=settings['loss'])

batch_size = inception_v3['batch_size']
model.fit_generator(train_generator_loaded,
                    steps_per_epoch= 0.1 * nb_train_examples / batch_size,
                    epochs=100, validation_data=valid_generator_loaded,
                    validation_steps = nb_valid_examples / batch_size,
                    callbacks = [model_checkpoint, es], verbose=1, initial_epoch = 34)

Epoch 35/100
216/215 [==============================] - 562s - loss: 0.0883 - val_loss: 0.0885
Epoch 36/100
216/215 [==============================] - 530s - loss: 0.0879 - val_loss: 0.0885
Epoch 37/100
216/215 [==============================] - 536s - loss: 0.0879 - val_loss: 0.0886
Epoch 38/100
216/215 [==============================] - 532s - loss: 0.0874 - val_loss: 0.0887
Epoch 00037: early stopping


In [ ]:
# define config for training

inception_v3 = {'name':'inception_v3', 'last_activation':'sigmoid',
                'output_size': 17, 'loss':'binary_crossentropy', 'input_shape': (299,299,3)}

# possible optimisers
rmsprop = model.compile(optimizer='rmsprop', loss=inception_v3['loss'], metrics=['accuracy'])
sgd_1 = model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=inception_v3['loss'], metrics=['accuracy'])
sgd_2 = model.compile(optimizer=SGD(lr=0.003, momentum=0.9), loss=inception_v3['loss'], metrics=['accuracy'])
sgd_3 = model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=inception_v3['loss'], metrics=['accuracy'])
sgd_3 = model.compile(optimizer=SGD(lr=0.0003, momentum=0.9), loss=inception_v3['loss'], metrics=['accuracy'])

inception_v3_p1 = {'phase':'top_layer', 'batch_size': 128, 'optimiser': rmsprop}
inception_v2_p2 = {'phase': 'full_1', 'batch_size': 16, 'optimiser': sgd_1,
                   'from_saved_model':'inception_v3_full_20_0.10.hdf5', initial_epoch: 20}

# F2 score on validation set

In [7]:
from keras.models import load_model

model = load_model('../models/inception_v3_full_37_0.09.hdf5')




In [8]:
p_valid = model.predict_generator(valid_generator_loaded, 6000/16, verbose=1)

375/375 [==============================] - 111s   


In [13]:
from sklearn.metrics import fbeta_score
simple_score = fbeta_score(y_valid[:6000], np.array(p_valid) > 0.2, beta=2, average='samples')
print(simple_score)

def get_optimal_threshhold(true_label, prediction, iterations = 100):

    best_threshhold = [0.2]*17    
    for t in range(17):
        best_fbeta = 0
        temp_threshhold = [0.2]*17
        for i in range(iterations):
            temp_value = i / float(iterations)
            temp_threshhold[t] = temp_value
            temp_fbeta = fbeta_score(true_label, prediction > temp_threshhold, beta=2, average='samples')
            if  temp_fbeta > best_fbeta:
                best_fbeta = temp_fbeta
                best_threshhold[t] = temp_value
    return best_threshhold
                
best_thrshold = get_optimal_threshhold(y_valid[:6000], p_valid, iterations = 100)

opti_score = fbeta_score(y_valid[:6000], np.array(p_valid) > best_thrshold, beta=2, average='samples')

print(simple_score)
print(opti_score)
print(best_thrshold)

0.92250005651


/usr/lib64/python3.4/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.92250005651
0.92706463798
[0.2, 0.24, 0.18, 0.1, 0.16, 0.11, 0.27, 0.05, 0.1, 0.22, 0.26, 0.1, 0.32, 0.28, 0.22, 0.33, 0.26]
